## Application of inter linear programming (ILP)
Use ILP to assign students into curses while maximizing student satisfiaction based on their preference lists.

In [4]:
import csv  
#import pulp to call GLPK, an engine to solve ILP
import pulp 
import re

# Create functions that read and write CSV file

In [5]:
#create a function called "read_in_csv_to_list" that reads csv 
def read_in_csv_to_list(filename, headers=False,only_row=None):
    start_row = 0
    if headers:
        start_row = 1
    with open(filename, 'r') as f:
        data = [row for row in csv.reader(f.read().splitlines())]
        
        if only_row == None:
            return data[start_row:]
        else:
            return data[only_row]

In [6]:
#define a function that writes csv
def write_list_to_csv(filename, data, headers=None):
    with open(filename, 'w') as csvfile:
        csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        if headers:
            csv_writer.writerow(headers)
        for row in data:
            csv_writer.writerow(row)

# Load csv file that contains each student's major and five choices after converting the csv format

In [9]:
#load csv file that contains each student's POE and five choices
student_choice_data = read_in_csv_to_list('StudentSurveyData.csv', headers=True) 
print(student_choice_data[:2])

[['', 'SOCIAL STUDIES_ SOCIAL WORK_ SOCIOLOGY', 'course_21', 'course_1', 'course_9', 'course_19', 'course_3'], ['', 'COMMUNICATION_ HEALTH COMMUNICATION__ THEATRE', 'course_17', 'course_9', 'course_2', 'course_1', 'course_16']]


## Load csv file that contains all courses 

In [29]:
#define them as courses_data
courses_data_raw = read_in_csv_to_list('CourseList.csv', headers=False, only_row=0)

#clean course names
COURSE_LIST = [re.sub('(^\W+)|(\W+$)','',n) for n in courses_data_raw]
print(COURSE_LIST)

['course_0', 'course_1', 'course_2', 'course_3', 'course_4', 'course_5', 'course_6', 'course_7', 'course_8', 'course_9', 'course_10', 'course_11', 'course_12', 'course_13', 'course_14', 'course_15', 'course_16', 'course_17', 'course_18', 'course_19', 'course_20', 'course_21', 'course_22', 'course_23', 'course_24']


## Check the number of students and courses 

In [30]:
NUM_STUDENTS = len(student_choice_data)
NUM_COURSE = len(COURSE_LIST)

#there is only one session in our study 
NUM_PERIODS = 1 

print('The number of students is {}'.format(NUM_STUDENTS))
print('The number of courses is {}'.format(NUM_COURSE))

The number of students is 361
The number of courses is 25


## Define minimum and maximum numbers of students in a course and alpha of the objective function

In [31]:
#define the minimum number of students in a course 
MIN_NUM_IN_COURSE = 14

#define the maximum number of students in a course 
MAX_NUM_IN_COURSE = 16

#define alpha of the objective function 
PARA_OBJECTIVE = 10

print('The min number of course size is {}'.format(MIN_NUM_IN_COURSE))
print('The max number of course size is {}'.format(MAX_NUM_IN_COURSE))
print('The alpha of objective function is {}'.format(PARA_OBJECTIVE))

The min number of course size is 14
The max number of course size is 16
The alpha of objective function is 10


## Load csv file that contains all POEs 

In [32]:
poe_data_raw = read_in_csv_to_list('MajorsList.csv', headers=False, only_row=0)
POE_LIST = [re.sub('(^\W+)|(\W+$)','',n) for n in poe_data_raw]
print(POE_LIST)

['BioBiochemChem', 'ArtsHumanities', 'SocialScience', 'Engineering', 'EnvironmentalRealted', 'Exploratory', 'OR Engineering EnvironmentalRealted']


## Load csv file that contains the maximum number of students in each course that can have the same major

In [33]:
poemax_data_raw = read_in_csv_to_list('MajorMax.csv', headers=False, only_row=0)
POEMAX_STRING_LIST = [re.sub('(^\W+)|(\W+$)','',n) for n in poemax_data_raw]
POEMAX_LIST = list(map(int, POEMAX_STRING_LIST))
print(POEMAX_LIST)

[6, 6, 4, 4, 4, 4, 10]


In [34]:
for major, max_num in zip(POE_LIST, POEMAX_LIST):
    print(f"Max_num student major in {major} is {max_num}")

Max_num student major in BioBiochemChem is 6
Max_num student major in ArtsHumanities is 6
Max_num student major in SocialScience is 4
Max_num student major in Engineering is 4
Max_num student major in EnvironmentalRealted is 4
Max_num student major in Exploratory is 4
Max_num student major in OR Engineering EnvironmentalRealted is 10


# Give a courseID to each course 

In [35]:
cources_dict = {cource: n for n, cource in enumerate(COURSE_LIST)}
cources_dict_reverse = {n: cource for n, cource in enumerate(COURSE_LIST)}

#print("'the name of course' : course ID = \n {} \n".format(cources_dict))
#print("course ID : 'the name of course' = \n {}".format(cources_dict_reverse))

# Store the information of each student in a useful format 

In [38]:
#define the placehold for the information of all students 
student_info_dicts = {}

for i, row in enumerate(student_choice_data):
    name, major, choice1, choice2, choice3, choice4, choice5 = row
    choices = [choice1, choice2, choice3, choice4, choice5]
    student_info_dicts[i] = {'choices': [cources_dict[c] for c in choices],
                             'choices_full_names': choices,
                             'name': name,
                             'major': major,
                             'assignment': []}

#print("studentID : 'choices':[], 'choices_full_names':[], 'name':'', 'major':'', 'assignment':[] = \n{}".format(student_info_dicts))

# Set up the ILP problem 

In [39]:
#want to maximize the value of the objective function 
prob = pulp.LpProblem("StudentClass", pulp.LpMaximize) 

#if we want to minimize the value of the objection function, use pulp.LpMaximize
#prob2 = pulp.LpProblem("StudentClass", pulp.LpMaximize) 


# Define decision variables 

In [40]:
#create a placeholder to store the list of coordinates for decision variables 
decision_var_matrix = []
for i in range(NUM_STUDENTS):
    for j in range(NUM_COURSE):
        decision_var_matrix.append((i, j))
#print("list of decision variables formed as (studentID, courseID) =  : \n {}".format(decision_var_matrix))

In [41]:
#create a placeholder to store the decision variables 
decision_vars_list = []
for i in range(NUM_PERIODS):
    variable_type_name = 'period_%s_decision_variable' % (i + 1)
    decision_vars_list.append(pulp.LpVariable.dicts(name=variable_type_name,
                                                    indexs=decision_var_matrix,
                                                    lowBound=0,
                                                    upBound=1,
                                                    cat=pulp.LpInteger))
#print(decision_vars_list)


### All decision variables are binary 0 or 1. If one decision varialbe (studentID, courseID) has the value of 1, then studentID is assigned into the corresponding courseID. The value of 0 means that studentID is not assigned into the corresponding courseID. 
### For example, (1, 11) = 1. It means that a student whose studentID is 1 is assigned into the course whose courseID is 11. (5, 9) = 0 means that a student whose studentID is 5 is not assigned into the course whose courseID is 9.


# Define constraints 

## CONSTRAINT 1: each student must be assigned into only one course 

In [42]:
for i in range(NUM_STUDENTS):
    for k in range(NUM_PERIODS):
        vars_to_sum = [decision_vars_list[k][(i, j)] for j in range(NUM_COURSE)]
        prob += pulp.lpSum(vars_to_sum) == 1 


### Consider one student whose studentID is 0. If there are ten courses offered by JC, then 
### (0, 0) + (0, 1) + (0, 2) + (0, 3) + (0, 4) + (0, 5) + (0, 6) + (0, 7) +(0, 8) + (0, 9) = 1
### If this student is assigned into the courseID 5, then (0, 5) = 1, and the rest of all decision variables are equal to 0. 

## CONSTRAINT 2: each course must have a minimum number of students

In [43]:
for j in range(NUM_COURSE):
    for k in range(NUM_PERIODS):
        vars_to_sum = [decision_vars_list[k][(i, j)] for i in range(NUM_STUDENTS)]
        prob += pulp.lpSum(vars_to_sum) >= MIN_NUM_IN_COURSE 
        

### Consider a course whose studentID is 5, and the mininum number of students in one course is 10. If there are three hundred students, then 
### (0, 5) + (1, 5) + (2, 5) + (3, 5) + ... + (298, 5) + (299, 5) >= 10.
### If studentID 1, 4, 34, 78, 134, 166, 204, 210, 250,  289, 290, and 295 are in the courseID 5, then (1, 5) + (4, 5) + (34, 5) + (78, 5) + (134, 5) + (166, 5) + (204, 5) + (210, 5) + (250, 5) + (289, 5) +  (290, 5) + (295, 5) = 12, where (1, 5) = (4, 5) = (34, 5) = (78, 5) = (134, 5) = (166, 5) = (204, 5) = (210, 5) = (250, 5) = (289, 5) =  (290, 5) = (295, 5) = 1.

## CONSTRAINT 3: each course can only have up to a maximum number of students

In [44]:
for j in range(NUM_COURSE):
    for k in range(NUM_PERIODS):
        vars_to_sum = [decision_vars_list[k][(i, j)] for i in range(NUM_STUDENTS)]
        prob += pulp.lpSum(vars_to_sum) <= MAX_NUM_IN_COURSE #prob = linear constraint  


### Consider a course whose studentID is 5, and the maximum number of students in one course is 15. If there are three hundred students, then 
### (0, 5) + (1, 5) + (2, 5) + (3, 5) + ... + (298, 5) + (299, 5) <= 15.
### If studentID 1, 4, 34, 78, 98, 134, 166, 204, 210, 250,  289, 290, and 295 are in the courseID 5, then (1, 5) + (4, 5) + (34, 5) + (78, 5) + (98, 5) +  (134, 5) + (166, 5) + (204, 5) + (210, 5) + (250, 5) + (289, 5) +  (290, 5) + (295, 5) = 12, where (1, 5) = (4, 5) = (34, 5) = (78, 5) = (98, 5) = (134, 5) = (166, 5) = (204, 5) = (210, 5) = (250, 5) = (289, 5) =  (290, 5) = (295, 5) = 1.

## CONSTRAINT 4: define the maximum number of the same major students in one course and also the maximum number of several specific major students in one course

In [45]:
for c in range(NUM_COURSE):
    for p in range(NUM_PERIODS):
        vars_to_sum = {}
               
        for poe in POE_LIST:
            vars_to_sum[poe] = []
            
        
        #update vars to sum for each poe
        for s in range(NUM_STUDENTS):
            major = student_info_dicts[s]['major']
            dvar = decision_vars_list[p][(s,c)]
            
            
            for poe in POE_LIST:
                if (poe == major) or (poe.find('OR ') == 0 and poe.find(major) != -1):
                    vars_to_sum[poe].append(dvar)
        
        for poe in range(len(POE_LIST)):
            #print('add constraint for ' + POES[poe] + '=' + str(MAX_STUDENTS_BY_POE[poe]) + ' in ' + str(c))
            prob += pulp.lpSum(vars_to_sum[POE_LIST[poe]]) <= POEMAX_LIST[poe] 
            
#print(prob)

## Constraint 5: All students are assigned into one of their five preferences 

In [19]:
for i in range(NUM_STUDENTS):
    vars_to_sum = [decision_vars_list[k][(i, j)] for j in student_info_dicts[i]['choices']
                                                 for k in range(NUM_PERIODS)]
    prob += pulp.lpSum(vars_to_sum) == 1 
    
#print("Make sure that each student will be assigned to one of their four chocies : \n {}".format(prob))

## Define the objective function 

In [46]:
objective_function_parts = []
for i in range(NUM_STUDENTS):
    for k in range(NUM_PERIODS):
        choice1, choice2, choice3, choice4, choice5 = student_info_dicts[i]['choices']
        objective_function_parts.append([decision_vars_list[k][(i, choice1)]*(PARA_OBJECTIVE**4)])
        objective_function_parts.append([decision_vars_list[k][(i, choice2)]*(PARA_OBJECTIVE**3)])
        objective_function_parts.append([decision_vars_list[k][(i, choice3)]*(PARA_OBJECTIVE**2)])
        objective_function_parts.append([decision_vars_list[k][(i, choice4)]*(PARA_OBJECTIVE**1)])
        objective_function_parts.append([decision_vars_list[k][(i, choice5)]*1])


prob += pulp.lpSum(objective_function_parts)   
#print("Want max of (student_ID, activity_ID[choices][0])*1000 + (student_ID, activity_ID[choices][1])*100 + (student_ID, activity_ID[choices][2])*10 + (student_ID, activity_ID[choices][3])*1 : \n {}".format(prob))

## Solve the ILP problem 

In [47]:
solution_found = prob.solve(pulp.GLPK())

#Anything other than solution_found == 1 indicates some issue with finding a solution that fits constraints)
assert solution_found == 1 #"solution_glpk not found"

print("Objective value:", pulp.value(prob.objective))
print(pulp.LpStatus[prob.status])

Objective value: 3122200
Optimal


## Write the final result in CSV 

### Append the assigned course to each student 

In [48]:
for period in range(NUM_PERIODS):
    for i in range(NUM_STUDENTS):
        for j in range(NUM_COURSE):
            if decision_vars_list[period][(i,j)].varValue == 1:
                student_info_dicts[i]['assignment'].append(cources_dict_reverse[j])
                #print(student_info_dicts[i]['assignment'][0])

### Create CSV that contains the column showing the assigned course for each student 

In [49]:
results = []
for i in range(NUM_STUDENTS):
    student_dict = student_info_dicts[i]
    name = student_dict['name']
    major = student_dict['major']
    choices = student_dict['choices_full_names']
    choice1, choice2, choice3, choice4, choice5 = choices
    assignment = student_dict['assignment'][0]
    row = [name, major, choice1, choice2, choice3, choice4, choice5, assignment]
    results.append(row)

headers = ['name', 'major', 'choice1', 'choice2', 'choice3', 'choice4', 'choice5',
           'assignment']

write_list_to_csv('Result.csv', data=results, headers=headers)

## Check the final outcome
### Show the number of total students who are assigned into their first, second, third, and fourth choice 

In [50]:
total_got_choice1 = 0
total_got_choice2 = 0
total_got_choice3 = 0
total_got_choice4 = 0
total_got_choice5 = 0


for i in range(NUM_STUDENTS):
    #print(student_info_dicts[i]['choices_full_names'])
    choices = student_info_dicts[i]['choices_full_names']
    #print(choices)
    choice1, choice2, choice3, choice4, choice5 = choices
    #print(student_info_dicts[i]['assignments'])
    #print(student_info_dicts[i]['assignments'])
    assignments = student_info_dicts[i]['assignment']
    
    #print(assignments)
    
    got_choice1, got_choice2, got_choice3, got_choice4, got_choice5  = 0, 0, 0, 0, 0
    if choice1 in assignments: got_choice1 = 1
    if choice2 in assignments: got_choice2 = 1
    if choice3 in assignments: got_choice3 = 1
    if choice4 in assignments: got_choice4 = 1
    if choice5 in assignments: got_choice5 = 1
        
    total_got_choice1 += got_choice1
    total_got_choice2 += got_choice2
    total_got_choice3 += got_choice3
    total_got_choice4 += got_choice4
    total_got_choice5 += got_choice5


print(total_got_choice1)
print(total_got_choice2)
print(total_got_choice3)
print(total_got_choice4)
print(total_got_choice5)

print("percent who got choice 1:", float(total_got_choice1) / NUM_STUDENTS * 100)
print("percent who got choice 2:", float(total_got_choice2) / NUM_STUDENTS * 100)
print("percent who got choice 3:", float(total_got_choice3) / NUM_STUDENTS * 100)
print("percent who got choice 4:", float(total_got_choice4) / NUM_STUDENTS * 100)
print("percent who got choice 5:", float(total_got_choice5) / NUM_STUDENTS * 100)

307
52
2
0
0
percent who got choice 1: 85.0415512465374
percent who got choice 2: 14.40443213296399
percent who got choice 3: 0.554016620498615
percent who got choice 4: 0.0
percent who got choice 5: 0.0
